In [1]:
import pickle as pkl
import numpy as np
from tqdm import tqdm
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments, pipeline
from nltk.tokenize import sent_tokenize, word_tokenize
import multiprocessing
import nltk
nltk.download('punkt')

/home/haskari/miniconda3/envs/acl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/haskari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
with open('data/cnn.pkl', 'rb') as f:
    cnn_chat=pkl.load(f)

In [4]:
#number of empty summaries
fail_count=0
for items in cnn_chat:
    if not items:
        fail_count+=0

print(fail_count)

0


In [6]:
count=0
for items in cnn_chat:
    items=nltk.sent_tokenize(items)

    if not items:
        continue
    elif len(items)==3:
        count+=1

print(count)

2489


In [7]:
print(len(cnn_chat))

11490


In [8]:
data = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'article'
summary_key = 'highlights'

data=data['test']

In [14]:
def tokenize(example):

    example["original_article"] = example[article_key]
    example["article"] = nltk.sent_tokenize(example[article_key])
    # if args.dataset == 'news':
    # check=example[summary_key][0]['text']
    # example["highlights"] = nltk.sent_tokenize(check)
    # else:
    example["highlights"] = nltk.sent_tokenize(example[summary_key])
        
    return example

data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

In [10]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

43560


In [11]:
count=0
for summ in cnn_chat:
    count+=len(nltk.sent_tokenize(summ))

print(count)
    

55553


In [8]:
with open('data/xsum_capped.pkl', 'rb') as f:
    xsum_chat=pkl.load(f)

In [9]:
#number of empty summaries
fail_count=0
for items in xsum_chat:
    if not items:
        fail_count+=0

print(fail_count)

0


In [11]:
count=0
for items in xsum_chat:
    #items=nltk.sent_tokenize(items)
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

11334


In [12]:
data = load_dataset("xsum")
article_key = 'document'
summary_key = 'summary'

data=data['test']

In [15]:
data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

In [16]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

11335


In [18]:
count=0
for summ in xsum_chat:
    count+=len(summ)

print(count)

11334


In [26]:
with open('data/news_capped.pkl', 'rb') as f:
    news_chat=pkl.load(f)

In [27]:
news_chat[0]

['The Swedish government has halted new development aid to Cambodia and will no longer support a reform programme initiated by the opposition party.']

In [28]:
fail_count=0
for items in news_chat:
    if not items:
        fail_count+=0

print(fail_count)

0


In [29]:
count=0
for items in news_chat:
    #items=nltk.sent_tokenize(items)
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

1000


In [30]:
data = load_dataset("argilla/news-summary")
article_key = 'text'
summary_key = 'prediction'
data = DatasetDict({
    'train': data['test'],
    'test': data['train']})

data=data['test']

In [31]:
def tokenize_news(example):
    example["original_article"] = example[article_key]
    example["article"] = nltk.sent_tokenize(example[article_key])

    check=example[summary_key][0]['text']
    example["highlights"] = nltk.sent_tokenize(check)

    return example

In [32]:
data = data.map(tokenize_news, num_proc=multiprocessing.cpu_count())

In [33]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

1012


In [35]:
count=0
for summ in news_chat:
    count+=len(summ)

print(count)

1000


In [37]:
with open('data/reddit_capped.pkl', 'rb') as f:
    reddit_chat=pkl.load(f)

In [38]:
#number of empty summaries
fail_count=0
for items in reddit_chat:
    if not items:
        fail_count+=0

print(fail_count)

0


In [40]:
count=0
for items in reddit_chat:
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

4214


In [41]:
data = load_dataset('reddit_tifu', 'long')
article_key = 'documents'
summary_key = 'tldr'
    # 80% train, 20% test + validation
train_testvalid = data['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
data = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

data=data['test']

In [42]:
data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

In [43]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

6016


In [44]:
count=0
for summ in reddit_chat:
    count+=len(summ)

print(count)

4214
